<a href="https://colab.research.google.com/github/ihatestudysomuch/pytorch/blob/main/Pytorch_6%EC%9E%A5_%EB%8B%A4%EC%A4%91%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 다중선형회귀 에제


-   앞서 말한 선형회귀 방법으로 pytorch 모델 구축



In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np

loaded_data = np.loadtxt('/content/drive/MyDrive/pytorch/PyTorch/LEC06_TrainData.csv', delimiter=',')

x_train_np = loaded_data[ : , 0:-1]

y_train_np = loaded_data[ : , [-1]]

print(loaded_data[:3])
print('========================')
print(x_train_np[:3])
print('========================')
print(y_train_np[:3])

[[ 1.  2.  0. -4.]
 [ 5.  4.  3.  4.]
 [ 1.  2. -1. -6.]]
[[ 1.  2.  0.]
 [ 5.  4.  3.]
 [ 1.  2. -1.]]
[[-4.]
 [ 4.]
 [-6.]]


In [8]:
import torch
from torch import nn

x_train = torch.Tensor(x_train_np)
y_train = torch.Tensor(y_train_np)

print(x_train)
print(y_train)

tensor([[ 1.,  2.,  0.],
        [ 5.,  4.,  3.],
        [ 1.,  2., -1.],
        [ 3.,  1.,  0.],
        [ 2.,  4.,  2.],
        [ 4.,  1.,  2.],
        [-1.,  3.,  2.],
        [ 4.,  3.,  3.],
        [ 0.,  2.,  6.],
        [ 2.,  2.,  1.],
        [ 1., -2., -2.],
        [ 0.,  1.,  3.],
        [ 1.,  1.,  3.],
        [ 0.,  1.,  4.],
        [ 2.,  3.,  3.]])
tensor([[-4.],
        [ 4.],
        [-6.],
        [ 3.],
        [-4.],
        [ 9.],
        [-7.],
        [ 5.],
        [ 6.],
        [ 0.],
        [ 4.],
        [ 3.],
        [ 5.],
        [ 5.],
        [ 1.]])


# 신경망 모델 구축

In [12]:
from torch import nn

class MyLinearRegressionModel(nn.Module):

    def __init__(self, input_nodes):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(input_nodes, 1) # nn.Linear 계층의 입력 노드 갯수, 출력 데이터 갯수 -> (inputn_nodes, 1)
        )

    # 피드 포워드
    def forward(self, data):
        prediction = self.linear_stack(data)

        return prediction

In [13]:
model = MyLinearRegressionModel(3) # 여기서 입력 데이터 3개인 선형회귀 모델 생성

for name, child in model.named_children():
    for param in child.parameters():
        print(name, param)   # 가중치 W, 바이어스 b 확인

linear_stack Parameter containing:
tensor([[-0.5013, -0.4936,  0.2963]], requires_grad=True)
linear_stack Parameter containing:
tensor([-0.0519], requires_grad=True)


# 손실함수 및 옵티마이저 정의

In [15]:
loss_function = nn.MSELoss() # 선형회귀이기 때문에 MSELoss 사용
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # SGD(확률적 경사 하강법) 사용

In [16]:
loss_list = []

# 2000번 학습
for epoch in range(2001):
    # prediction 즉, 학습 하는 과정에서 forward() 함수 자동 호출
    prediction = model(x_train)
    # 선형회귀 모델의 예측값(x_train으로 학습한 prediction)과 정답과의 오차 계산
    loss = loss_function(prediction, y_train)

    # 역전파(backpropagation)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch : {epoch}, Loss : {loss.item()}')


Epoch : 0, Loss : 29.273530960083008
Epoch : 100, Loss : 0.13605180382728577
Epoch : 200, Loss : 0.0014289248501881957
Epoch : 300, Loss : 0.00013353940448723733
Epoch : 400, Loss : 4.237889152136631e-05
Epoch : 500, Loss : 1.4298474525276106e-05
Epoch : 600, Loss : 4.8308406803698745e-06
Epoch : 700, Loss : 1.6324976286341553e-06
Epoch : 800, Loss : 5.522173864846991e-07
Epoch : 900, Loss : 1.8683000746477774e-07
Epoch : 1000, Loss : 6.331328705755368e-08
Epoch : 1100, Loss : 2.148900435372525e-08
Epoch : 1200, Loss : 7.287266790001468e-09
Epoch : 1300, Loss : 2.487514860760598e-09
Epoch : 1400, Loss : 8.670537066102213e-10
Epoch : 1500, Loss : 3.1344910000896675e-10
Epoch : 1600, Loss : 1.283201184199001e-10
Epoch : 1700, Loss : 6.114408285240458e-11
Epoch : 1800, Loss : 3.608445486857903e-11
Epoch : 1900, Loss : 2.131352559719968e-11
Epoch : 2000, Loss : 1.9995496577940308e-11


실행 결과를 보면 알다시피 epoch 즉, 반복이 많이 될수록 Loss가 0에 수렴함

In [17]:
for name, child in model.named_children():
    for param in child.parameters():
        print(name, param)

linear_stack Parameter containing:
tensor([[ 2.0000, -3.0000,  2.0000]], requires_grad=True)
linear_stack Parameter containing:
tensor([6.3153e-06], requires_grad=True)


In [18]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 2.0000, -3.0000,  2.0000]], requires_grad=True)
Parameter containing:
tensor([6.3153e-06], requires_grad=True)


실행 결과를 보다시피 W(2,-3,2)에 잘 학습했고
bias = 0을 지향하기 때문에 거의 0에 수렴함을 알 수 있음

# test


In [19]:
x_test = torch.Tensor([ [5, 5, 0], [2, 3, 1], [-1, 0, -1], [10, 5, 2], [4, -1, -2] ])

label = [ 2*data[0] -3*data[1] + 2*data[2]  for data in x_test ] # 가중치와 test data로 정답 도출

pred = model(x_test) # test data로 정답과 비슷한지 확인

print(pred)
print('=============================================')
print(label)

tensor([[-5.0000],
        [-3.0000],
        [-4.0000],
        [ 9.0000],
        [ 7.0000]], grad_fn=<AddmmBackward0>)
[tensor(-5.), tensor(-3.), tensor(-4.), tensor(9.), tensor(7.)]


실행 결과 예측값과 정답값이 완전 일치